# Borda Count Voting Analysis

This notebook analyzes strategic manipulation in Borda count voting. It computes the winner under truthful voting and finds the best strategic deviation.


In [90]:
# Configuration and Constants

# Borda scoring: 1st place = 2 points, 2nd place = 1 point, 3rd place = 0 points
BORDA_SCORES = {0: 2, 1: 1, 2: 0}

# Voting profile (excluding the manipulator's 15 votes)
OTHER_VOTERS = {
    ('A', 'B', 'C'): 15,
    ('A', 'C', 'B'): 25,
    ('B', 'A', 'C'): 10,
    ('B', 'C', 'A'): 15,
    ('C', 'B', 'A'): 15,
}

# True preference of the manipulator
TRUE_PREFERENCE = ('C', 'A', 'B')
MANIPULATOR_VOTES = 20

# Utility mapping based on true preference C > A > B
UTILITY = {'C': 2, 'A': 1, 'B': 0}

print("✓ Configuration loaded")
print(f"  True preference: {' > '.join(TRUE_PREFERENCE)}")
print(f"  Manipulator controls: {MANIPULATOR_VOTES} votes")
print(f"  Other voters: {sum(OTHER_VOTERS.values())} votes")


✓ Configuration loaded
  True preference: C > A > B
  Manipulator controls: 20 votes
  Other voters: 80 votes


## Helper Functions


In [91]:
def compute_borda_scores(voter_profile):
    """
    Compute Borda scores for each candidate given a voter profile.
    
    Args:
        voter_profile: Dictionary mapping rankings (tuples) to number of voters
        
    Returns:
        Dictionary mapping candidates to their total Borda scores
    """
    scores = {'A': 0, 'B': 0, 'C': 0}
    
    for ranking, count in voter_profile.items():
        for position, candidate in enumerate(ranking):
            scores[candidate] += count * BORDA_SCORES[position]
    
    return scores


def determine_winner(scores):
    """
    Determine the winner based on Borda scores.
    In case of a tie, returns the candidate with lexicographically first name.
    
    Args:
        scores: Dictionary mapping candidates to their Borda scores
        
    Returns:
        The winning candidate
    """
    max_score = max(scores.values())
    winners = [cand for cand, score in scores.items() if score == max_score]
    return sorted(winners)[0]  # Lexicographic tie-breaking


def compute_outcome(manipulator_ranking):
    """
    Compute the election outcome given the manipulator's ranking.
    
    Args:
        manipulator_ranking: Tuple representing the manipulator's ranking (e.g., ('C', 'A', 'B'))
        
    Returns:
        Tuple of (winner, scores_dict)
    """
    # Create full profile including manipulator
    full_profile = OTHER_VOTERS.copy()
    full_profile[manipulator_ranking] = full_profile.get(manipulator_ranking, 0) + MANIPULATOR_VOTES
    
    scores = compute_borda_scores(full_profile)
    winner = determine_winner(scores)
    
    return winner, scores


def get_all_rankings():
    """
    Generate all possible rankings of three candidates.
    
    Returns:
        List of tuples representing all 6 possible rankings
    """
    from itertools import permutations
    candidates = ['A', 'B', 'C']
    return list(permutations(candidates))

print("✓ Helper functions defined")


✓ Helper functions defined


## Truthful Voting Analysis


In [92]:
# Compute truthful outcome
truthful_winner, truthful_scores = compute_outcome(TRUE_PREFERENCE)
truthful_utility = UTILITY[truthful_winner]

print("=" * 80)
print("TRUTHFUL VOTING")
print("=" * 80)
print(f"Ranking: {' > '.join(TRUE_PREFERENCE)}")
print(f"\nBorda scores:")
for candidate, score in sorted(truthful_scores.items(), key=lambda x: -x[1]):
    print(f"  {candidate}: {score} points")
print(f"\nWinner: {truthful_winner}")
print(f"Utility (based on C > A > B): {truthful_utility}")
print(f"  (C=2, A=1, B=0)")


TRUTHFUL VOTING
Ranking: C > A > B

Borda scores:
  A: 110 points
  C: 110 points
  B: 80 points

Winner: A
Utility (based on C > A > B): 1
  (C=2, A=1, B=0)


## Testing All Possible Deviations


In [93]:
# Test all possible rankings
all_rankings = get_all_rankings()
all_results = []

print("=" * 80)
print("TESTING ALL POSSIBLE RANKINGS")
print("=" * 80)

for ranking in all_rankings:
    winner, scores = compute_outcome(ranking)
    utility = UTILITY[winner]
    
    result = {
        'ranking': ranking,
        'winner': winner,
        'scores': scores,
        'utility': utility
    }
    all_results.append(result)
    
    ranking_str = ' > '.join(ranking)
    is_truthful = ranking == TRUE_PREFERENCE
    marker = " [TRUTHFUL]" if is_truthful else ""
    
    print(f"\nRanking: {ranking_str}{marker}")
    print(f"  Scores: A={scores['A']}, B={scores['B']}, C={scores['C']}")
    print(f"  Winner: {winner} (utility: {utility})")


TESTING ALL POSSIBLE RANKINGS

Ranking: A > B > C
  Scores: A=130, B=100, C=70
  Winner: A (utility: 1)

Ranking: A > C > B
  Scores: A=130, B=80, C=90
  Winner: A (utility: 1)

Ranking: B > A > C
  Scores: A=110, B=120, C=70
  Winner: B (utility: 0)

Ranking: B > C > A
  Scores: A=90, B=120, C=90
  Winner: B (utility: 0)

Ranking: C > A > B [TRUTHFUL]
  Scores: A=110, B=80, C=110
  Winner: A (utility: 1)

Ranking: C > B > A
  Scores: A=90, B=100, C=110
  Winner: C (utility: 2)


## Finding Best Strategic Deviation


In [94]:
# Find best deviation (excluding truthful ranking)
deviations = [r for r in all_results if r['ranking'] != TRUE_PREFERENCE]
best_deviation = max(deviations, key=lambda x: x['utility'])

print("=" * 80)
print("FINDING BEST STRATEGIC DEVIATION")
print("=" * 80)

print(f"\nTruthful outcome:")
print(f"  Ranking: {' > '.join(TRUE_PREFERENCE)}")
print(f"  Winner: {truthful_winner} (utility: {truthful_utility})")

print(f"\nBest deviation:")
print(f"  Ranking: {' > '.join(best_deviation['ranking'])}")
print(f"  Winner: {best_deviation['winner']} (utility: {best_deviation['utility']})")
print(f"  Scores: {best_deviation['scores']}")

if best_deviation['utility'] > truthful_utility:
    improvement = best_deviation['utility'] - truthful_utility
    print(f"\n✓ Strategic manipulation improves outcome by {improvement} utility!")
    print(f"  Can change winner from {truthful_winner} to {best_deviation['winner']}")
else:
    print(f"\n✗ No beneficial deviation found.")
    print(f"  Truthful voting is already optimal.")


FINDING BEST STRATEGIC DEVIATION

Truthful outcome:
  Ranking: C > A > B
  Winner: A (utility: 1)

Best deviation:
  Ranking: C > B > A
  Winner: C (utility: 2)
  Scores: {'A': 90, 'B': 100, 'C': 110}

✓ Strategic manipulation improves outcome by 1 utility!
  Can change winner from A to C


## Detailed Comparison


In [95]:
print("=" * 80)
print("ALL RANKINGS SORTED BY UTILITY")
print("=" * 80)

sorted_results = sorted(all_results, key=lambda x: -x['utility'])
for i, result in enumerate(sorted_results, 1):
    ranking_str = ' > '.join(result['ranking'])
    is_truthful = result['ranking'] == TRUE_PREFERENCE
    marker = " [TRUTHFUL]" if is_truthful else ""
    print(f"{i}. {ranking_str}{marker}")
    print(f"   Winner: {result['winner']}, Utility: {result['utility']}")


ALL RANKINGS SORTED BY UTILITY
1. C > B > A
   Winner: C, Utility: 2
2. A > B > C
   Winner: A, Utility: 1
3. A > C > B
   Winner: A, Utility: 1
4. C > A > B [TRUTHFUL]
   Winner: A, Utility: 1
5. B > A > C
   Winner: B, Utility: 0
6. B > C > A
   Winner: B, Utility: 0


## Interactive Exploration Functions


In [96]:
def explore_ranking(ranking_tuple):
    """
    Explore a specific ranking interactively.
    
    Usage:
        explore_ranking(('A', 'C', 'B'))
    """
    winner, scores = compute_outcome(ranking_tuple)
    utility = UTILITY[winner]
    
    print(f"\nExploring ranking: {' > '.join(ranking_tuple)}")
    print(f"  Scores: {scores}")
    print(f"  Winner: {winner}")
    print(f"  Utility: {utility}")
    
    if ranking_tuple == TRUE_PREFERENCE:
        print(f"  [This is the truthful ranking]")
    else:
        diff = utility - truthful_utility
        if diff > 0:
            print(f"  [Better than truthful by {diff} utility]")
        elif diff < 0:
            print(f"  [Worse than truthful by {abs(diff)} utility]")
        else:
            print(f"  [Same utility as truthful]")
    
    return winner, scores, utility


def compare_rankings(ranking1, ranking2):
    """
    Compare two rankings side by side.
    
    Usage:
        compare_rankings(TRUE_PREFERENCE, ('A', 'C', 'B'))
    """
    r1_winner, r1_scores, r1_util = explore_ranking(ranking1)
    r2_winner, r2_scores, r2_util = explore_ranking(ranking2)
    
    print(f"\nComparison:")
    print(f"  Ranking 1 ({' > '.join(ranking1)}): {r1_winner} wins, utility {r1_util}")
    print(f"  Ranking 2 ({' > '.join(ranking2)}): {r2_winner} wins, utility {r2_util}")
    
    if r2_util > r1_util:
        print(f"  → Ranking 2 is better by {r2_util - r1_util} utility")
    elif r1_util > r2_util:
        print(f"  → Ranking 1 is better by {r1_util - r2_util} utility")
    else:
        print(f"  → Both rankings yield the same utility")

print("✓ Interactive functions defined")
print("\nYou can now use:")
print("  - explore_ranking(('A', 'C', 'B'))")
print("  - compare_rankings(TRUE_PREFERENCE, ('A', 'C', 'B'))")
print("  - all_results  # List of all ranking results")
print("  - best_deviation  # Best strategic deviation")


✓ Interactive functions defined

You can now use:
  - explore_ranking(('A', 'C', 'B'))
  - compare_rankings(TRUE_PREFERENCE, ('A', 'C', 'B'))
  - all_results  # List of all ranking results
  - best_deviation  # Best strategic deviation


## Example: Explore Best Deviation


In [97]:
# Compare truthful vs best deviation
if best_deviation['utility'] > truthful_utility:
    compare_rankings(TRUE_PREFERENCE, best_deviation['ranking'])



Exploring ranking: C > A > B
  Scores: {'A': 110, 'B': 80, 'C': 110}
  Winner: A
  Utility: 1
  [This is the truthful ranking]

Exploring ranking: C > B > A
  Scores: {'A': 90, 'B': 100, 'C': 110}
  Winner: C
  Utility: 2
  [Better than truthful by 1 utility]

Comparison:
  Ranking 1 (C > A > B): A wins, utility 1
  Ranking 2 (C > B > A): C wins, utility 2
  → Ranking 2 is better by 1 utility
